In [1]:
from mmnl import MMNL
import numpy as np
import pickle
import pandas as pd


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Authorize Google Drive to upload files to the drive

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
def load_data(path):
    dat = np.load(path)
    X = dat[:, :-1]
    Y = np.reshape(dat[:, -1], (-1, 1))
    return X, Y
X, Y = load_data('data/data.npy')

Load data

In [3]:
%%time
infile = open('./data/500_QMC_dgp.p', 'rb')
big_dict = pickle.load(infile)
Y_dgp = big_dict['theta: [ 1.5  1.  -1.1  0.8  0.1  1.2]']
print(Y_dgp.shape)

(2798, 500)
Wall time: 25.3 ms


In [4]:
%%time
result = []
draws = 250
start = 0
end = 30
method = 'SMC'
for i in range(start,end):
    print('iteration: %i' %(i+1))
    qmc = MMNL(X,Y_dgp[:,i],draws,3,method=method)
    solution = qmc.solver()
    result.append(solution)
    if i%10==9:
        pickle.dump( result, open( "./resultaten/%s(%i)/%i-%i_%s(%i)_results2.p" %(method,draws,i-9,i,method,draws), "wb" ) )
        result = []
#     f = drive.CreateFile()
#     f.SetContentFile("./resultaten/%i-%i_%s(%i)_results.p" %(i,i+10,method,draws))
#     f.Upload()

iteration: 1


C:\Users\niels\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:502: RuntimeWarning: Method Nelder-Mead does not use gradient information (jac).
  RuntimeWarning)


 final_simplex: (array([[ 3.16966161e+00,  8.02387094e-01, -2.64386895e+00,
         2.37638258e+00, -1.24360943e-06, -4.53413476e-09],
       [ 3.16968206e+00,  8.02389187e-01, -2.64388616e+00,
         2.37646365e+00,  9.98218750e-08,  1.06332307e-07],
       [ 3.16966020e+00,  8.02411591e-01, -2.64386770e+00,
         2.37629492e+00,  2.83215233e-07,  2.02333976e-07],
       [ 3.16964038e+00,  8.02399263e-01, -2.64385128e+00,
         2.37638589e+00, -1.01793886e-06,  1.78907448e-07],
       [ 3.16967491e+00,  8.02400726e-01, -2.64387998e+00,
         2.37631794e+00, -6.29203564e-08, -2.80824107e-07],
       [ 3.16967810e+00,  8.02406560e-01, -2.64388284e+00,
         2.37642292e+00, -3.63874566e-07,  1.15991720e-07],
       [ 3.16961479e+00,  8.02415987e-01, -2.64382986e+00,
         2.37631596e+00, -6.95429681e-07, -3.18999687e-07]]), array([1801.86939973, 1801.86939973, 1801.86939982, 1801.86939982,
       1801.86939988, 1801.86939993, 1801.86939995]))
           fun: 1801.869399

KeyboardInterrupt: 

In [11]:
pickle.dump( result, open( "./resultaten/BQMC(8)/90_BQMC(8)_results.p", "wb" ) )

In [10]:
x_est = np.array(result)
x_est.shape

(100, 6)

In [11]:
np.mean(x_est, axis=0)

array([ 4.58777441,  2.65487859, -7.12572568,  7.45172249,  2.22983804,
        5.23629935])